In [1]:
# import
import torch
from tqdm import tqdm
from transformers import BertConfig, BertForPreTraining
# 標準使用ライブラリー
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')
import gc
import os
import shutil
from icecream import ic
from tqdm import tqdm_notebook as tqdm 

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# 追記
import json
import datetime
import math
plt.style.use('dark_background')

from torch import nn

from transformers import BertModel
import os
os.environ["CUDA_VISIBLE_DEVICES"]="9"


In [2]:
dic = {}

with open("../BERT-base_mecab-ipadic-bpe-32k/vocab.txt",  "r", encoding="utf-8" ) as f:
    vocab = f.read()
    for id, word in enumerate(vocab.split('\n')):
        dic[word] = id
        
text = "[CLS] 私 は 犬 が 好き 。 [SEP]"

x = [dic[w] for w in text.split()]

In [3]:
x = torch.LongTensor(x).unsqueeze(0)



In [4]:
bert = BertModel.from_pretrained("cl-tohoku/bert-base-japanese")

In [5]:
# a = bert(x)

In [6]:
# a[0].shape

In [7]:
from transformers import BertJapaneseTokenizer

tknz = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese')


In [8]:
tknz.tokenize("私は犬が好き")

['私', 'は', '犬', 'が', '好き']

# Masked Language Model

In [9]:
x_train, x_test = [],[]
y_train, y_test = [],[]


In [10]:
train_df = pd.read_csv("../data/train_twitterJSA_data.csv")

test_df = pd.read_csv("../data/test_twitterJSA_data.csv")

In [12]:
for ind in tqdm(range(len(train_df))):
#     print(ind)
    
    if type(train_df["text"].iloc[ind]  ) != str:
        continue
    tid = tknz.encode(train_df["text"].iloc[ind]) 
    
    if (len(tid)>512):
        tid = tid[:512]
    x_train.append(tid)
    y_train.append(train_df["label"].iloc[ind])
#     break

In [13]:
# net(x_train[1])

# net(torch.LongTensor(x_train[1]).unsqueeze(0).to(device))

In [14]:
for i in range(len(x_train)):
    if (len(x_train[i])) < 2:
        print(x_train[i])


In [15]:
for ind in tqdm(range(len(test_df))):
    
    if type(test_df["text"].iloc[ind] ) != str:
        continue
    
    tid = tknz.encode(test_df["text"].iloc[ind]) 
    if (len(tid)>512):
        tid = tid[:512]
    x_test.append(tid)
    y_test.append(test_df["label"].iloc[ind])
#     break

In [16]:
class DocCls(nn.Module):
    def __init__(self, bert):
        super(DocCls, self).__init__()
        self.bert = bert
        self.cls = nn.Linear(768, 5)
    
    def forward(self, x):
        bout = self.bert(x)
        bs = len(bout[0])
        h0 = [bout[0][i][0] for i in range(bs)]
        h0 = torch.stack(h0, dim=0)
        return self.cls(h0)
        
    

In [17]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [18]:
from torch import optim
net = DocCls(bert)
net.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr = 0.001)

## 学習

In [19]:
net.train()

for ep in tqdm(range(10)):
    lossK=0.0
    for i in range(len(x_train)):
        x= torch.LongTensor(x_train[i]).unsqueeze(0).to(device)
        dummy_y  =[]
        added_flg = 0
        
        # 同率１位は、先のほうにする。
        for index, yy in enumerate(y_train[i][1:-1].split(",")):
            if "1" in yy and added_flg == 0:
                dummy_y.append(index)
                added_flg = 1
            
        y = torch.LongTensor(dummy_y).to(device)
        out = net(x)
#         print("out:", out.size(), "y:", y.size())
        
        loss = criterion(out, y)
        lossK += loss.item()
        
        if (i % 2000 == 0 ):
            print(ep, i, lossK)
            lossK = 0.0
        elif (i>50000):
            break
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    outfile = "doccls - "+ str(ep) + ".model"
            

0 0 1.5006392002105713
0 2000 1869.4233348965645
0 4000 1816.6010485887527
0 6000 1735.1425457000732
0 8000 1621.5836631059647
0 10000 1613.6007076501846
0 12000 1533.5128128528595
0 14000 1454.4160539507866
0 16000 1411.877355158329
0 18000 1437.2804020643234
0 20000 1369.4372972249985
0 22000 1423.184942483902
0 24000 1355.9320271015167
0 26000 1318.4401175379753
0 28000 1353.7926314473152
0 30000 1313.5203562378883
0 32000 1308.4697924852371
0 34000 1356.1849667429924
0 36000 1252.3874206542969
0 38000 1326.4511721134186
0 40000 1310.6814140677452
0 42000 1252.5557119846344
0 48000 1276.300983607769
0 50000 1275.4331720471382
1 0 1.9945297241210938
1 2000 1233.008622944355
1 4000 1253.9622992873192
1 6000 1286.7773941755295
1 8000 1242.713795542717
1 10000 1279.1937818527222
1 12000 1224.172103881836
1 14000 1177.4856850504875
1 16000 1156.9402059316635
1 18000 1175.6024389863014
1 20000 1125.7209657430649
1 22000 1182.1059564948082
1 24000 1145.7374956011772
1 26000 1084.7293986082

In [20]:
torch.save(net.state_dict(), outfile)




## 評価

In [55]:
real_data_num, ok =0, 0
net.eval()

with torch.no_grad():
    for i in range(len(x_test)):
        x = torch.LongTensor(x_test[i]).unsqueeze(0).to(device)
        ans =  net(x)
        ans1 = torch.argmax(ans,dim=1).item()
        
        if (ans1 ==  [int(a) for a in y_test[i][1:-1].split(",")].index(1)  ):
            ok += 1
        real_data_num += 1

print(ok, real_data_num, ok/real_data_num)


45187 61064 0.7399941045460501
